In [1]:
#import libiary
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

#import requests
#from bs4 import BeautifulSoup
#import re
#import matplotlib.pyplot as plt

In [2]:
#pip install holidays

Note: you may need to restart the kernel to use updated packages.


In [3]:
import holidays

us_holidays = holidays.US(years=2019, prov='NERC')

print("US NERC Holidays:")
for date, name in sorted(us_holidays.items()):
    print(date, name)

US NERC Holidays:
2019-01-01 New Year's Day
2019-01-21 Martin Luther King Jr. Day
2019-02-18 Washington's Birthday
2019-05-27 Memorial Day
2019-07-04 Independence Day
2019-09-02 Labor Day
2019-10-14 Columbus Day
2019-11-11 Veterans Day
2019-11-28 Thanksgiving
2019-12-25 Christmas Day


### Assignment 1: Power Calendar function

**iso (character):** one of PJM/MISO/ERCOT/SPP/NYISO/WECC/CAISO (see item 1 below)  
**peak.type (character):** one of onpeak/offpeak/flat/2x16H/7x8  
**period (character):** has 4 types: “2018-2-3” as a daily, “2018Mar” as a monthly, “2018Q2” as a quarterly, “2018A” as an annually.

1. In US, there are 7 major ISOs: see https://www.ferc.gov/power-sales-and-markets/rtos-and-isos  
PJMISO, MISO, ERCOT, SPPISO, NYISO are eastern,   
WECC and CAISO is western.
2. HE, short for Hour Ending. HE2 means 01:00 - 02:00, HE14 means 13:00 - 14:00. For each single day, we have HE1 to HE24.
3. peak.type, eastern power market considers a HE to be onpeak, if it's a non-NERC holiday weekday from HE7 to HE22, the rest are offpeak HEs. If the peak.type is flat, that means every hour. (Hint, R package “timeDate” gives NERC holidays). 2x16H is HE7 to HE22 for the weekend and the NERC holiday. 7x8 is non HE7 to HE22 through the week.
4. Western market accepts all the assumptions from Eastern, moreover, it takes Saturday as a weekday.
5. MISO does not have the daylight-saving setting, the rest have. (Hint: daylight-savings will impact the function for certain month/peak.type.)

In [4]:
#define function:
def get_hours(iso, peak_type, period):
    # Process Period: period (character): has 4 types
    ##define each input by length:
    period_length = len(period)
    
    #check first 4 characters:
    if not period[:4].isdigit():
        raise ValueError("This is a Wrong Input,\n Input instruction: \n“2018-2-3” as a daily, “2018Mar” as a monthly, “2018Q2” as a quarterly, “2018A” as an annually.")
    
    #Define year
    year = int(period[:4])

    #Annually
    if period_length == 5: 
        start_date = datetime(year, 1, 1)
        end_date = datetime(year, 12, 31)
    
    #Quarterly
    elif period_length == 6: 
        quarter = int(period[5])
        
        start_month = 3 * (quarter - 1) + 1
        start_date = datetime(year, start_month, 1)
        
        end_month = start_month + 2
        end_date = (datetime(year, end_month + 1, 1) - timedelta(days=1))

    #Monthly
    elif period_length == 7: 
        month = datetime.strptime(period[4:], "%b").month
        start_date = datetime(year, month, 1)
        end_date = (datetime(year, month + 1, 1) - timedelta(days=1))

    # Daily
    elif period_length > 7 and period[-1].isdigit():
        start_date = datetime.strptime(period, "%Y-%m-%d") #phrase to datetime format
        end_date = start_date
    else:
        raise ValueError("This is a Wrong Input,\n Input instruction: \n“2018-2-3” as a daily, “2018Mar” as a monthly, “2018Q2” as a quarterly, “2018A” as an annually.")

    # Process peak type
    ## peak.type (character): one of onpeak/offpeak/flat/2x16H/7x8
    
    #daylight saving indicator
    include = 0 
    days = pd.date_range(start=start_date, end=end_date, freq='D')

    #Flat Peak for all day
    if peak_type == "flat":
        include = 1
        hours = len(days) * 24

    #everyday of a week, for 8 hours
    elif peak_type == "7x8":
        include = 1
        hours = len(days) * 8

    else:
        weekdays = days.weekday
        year_holidays = [datetime.strptime(date_str, "%Y-%m-%d") for date_str in ["2019-01-01", "2019-05-27", "2019-07-04", "2019-09-02", "2019-11-28", "2019-12-25"]]
        holidays = [d for d in year_holidays if start_date <= d <= end_date]
        non_holidays = [d for d in days if d not in holidays]

        # Process ISO
        if iso in ["PJMISO", "MISO", "ERCOT", "SPPISO", "NYISO"]:  # Eastern market
            weekend = [5, 6]
        elif iso in ["WECC", "CAISO"]:  # Western market
            weekend = [5]
        else:
            raise ValueError("Wrong input of iso")

        non_holiday_weekdays = [d for d in non_holidays if d.weekday() not in weekend]

        if peak_type == "onpeak":
            hours = len(non_holiday_weekdays) * 16

        elif peak_type == "offpeak":
            include = 1
            hours = len(days) * 24 - len(non_holiday_weekdays) * 16

        elif peak_type == "2x16H":
            weekend_holidays = list(set(holidays).union(set(days[weekdays.isin(weekend)])))
            hours = len(set(weekend_holidays)) * 16

    #Day light saving
    if iso != "MISO" and include == 1:  # only MISO not participate
        march_dates = pd.date_range(start=datetime(year, 3, 1), end=datetime(year, 3, 31))
        march_sundays = [d for d in march_dates if d.weekday() == 6]
        # Add daylight-saving time

    # Return the result
    result = {
        "iso": iso,
        "peak_type": peak_type,
        "start_date": start_date,
        "end_date": end_date,
        "num_hour": hours
    }
    return result


In [5]:
try1 = get_hours("ERCOT", "onpeak", "2019May")
try1

{'iso': 'ERCOT',
 'peak_type': 'onpeak',
 'start_date': datetime.datetime(2019, 5, 1, 0, 0),
 'end_date': datetime.datetime(2019, 5, 31, 0, 0),
 'num_hour': 352}